In [1]:
from transformers import AutoTokenizer

#加载编码器
tokenizer = AutoTokenizer.from_pretrained('distilgpt2', use_fast=True)

print(tokenizer)

#编码试算
tokenizer.batch_encode_plus([
    'hide new secretions from the parental units',
    'contains no wit , only labored gags'
])

PreTrainedTokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})


{'input_ids': [[24717, 649, 3200, 507, 422, 262, 21694, 4991], [3642, 1299, 645, 20868, 837, 691, 2248, 1850, 308, 3775]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [2]:
from datasets import load_dataset, load_from_disk

#加载数据
#dataset = load_dataset(path='glue', name='sst2')
dataset = load_from_disk('datas/glue/sst2')


#分词,同时删除多余的字段
def f(data):
    return tokenizer.batch_encode_plus(data['sentence'])


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['sentence', 'idx', 'label'])


#过滤掉太短的句子
def f(data):
    return [len(i) >= 8 for i in data['input_ids']]


dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)


#截断句子,同时整理成模型需要的格式
def f(data):
    data['input_ids'] = [i[:8] for i in data['input_ids']]
    data['attention_mask'] = [[1] * 8] * len(data['attention_mask'])
    #在模型中处理了偏移量问题,这里保持输入输出一致即可
    data['labels'] = data['input_ids'].copy()
    return data


dataset = dataset.map(f, batched=True, batch_size=1000, num_proc=4)

dataset, dataset['train'][0]

Loading cached processed dataset at datas/glue/sst2/train/cache-2fa3d9ea0c2e3200.arrow


Loading cached processed dataset at datas/glue/sst2/train/cache-cac77a5e86fd7743.arrow


Loading cached processed dataset at datas/glue/sst2/train/cache-3cc26b7057143977.arrow


Loading cached processed dataset at datas/glue/sst2/train/cache-8be3df80a5174f21.arrow


Loading cached processed dataset at datas/glue/sst2/validation/cache-5defa0b47ec6fdf7.arrow


...

Loading cached processed dataset at datas/glue/sst2/test/cache-e4318eba63d6359f.arrow


Loading cached processed dataset at datas/glue/sst2/test/cache-37a41e2a435574e7.arrow


Loading cached processed dataset at datas/glue/sst2/test/cache-74c7fa3e87d7b806.arrow


Loading cached processed dataset at datas/glue/sst2/test/cache-8a1840d9a979acde.arrow


(DatasetDict({
     train: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 39905
     })
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 848
     })
     test: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 1730
     })
 }),
 {'input_ids': [24717, 649, 3200, 507, 422, 262, 21694, 4991],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [24717, 649, 3200, 507, 422, 262, 21694, 4991]})

In [3]:
import torch
from transformers.data.data_collator import default_data_collator

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=default_data_collator,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

len(loader), data

(4988,
 {'input_ids': tensor([[10134,  1714,   319,  3159,   587,   523, 19855,  3098],
          [33587,  2879,   318,   262,  1266,   339,   705,    82],
          [ 2339,   427,  7274,   705,    82, 16924,   286,   262],
          [10134,  1716,   262,  4958,   286,  3527,    84, 31110],
          [  505,  1263, 12226,   284,   711,   530, 45179,  3715],
          [ 1136,   517,   302,    12,  7513,   602,   286,   477],
          [ 1462,   804, 45630,   272, 48973,   287,   262,  4151],
          [  986,   743,  1234,   572, 31594,   290, 29028, 12936]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1]]),
  'labels': tensor([[10134,  1714,   319,  3159,   587,   523, 19855,  3098],
          [33587,  2879,   318,   262,  1266,   

In [4]:
from transformers import AutoModelForCausalLM, GPT2Model

#加载模型
#model = AutoModelForCausalLM.from_pretrained('distilgpt2')


#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = GPT2Model.from_pretrained('distilgpt2')
        self.fc = torch.nn.Linear(768, tokenizer.vocab_size, bias=False)
        
        #加载预训练模型的参数
        parameters = AutoModelForCausalLM.from_pretrained('distilgpt2')
        self.fc.load_state_dict(parameters.lm_head.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        logits = self.pretrained(input_ids=input_ids,
                                 attention_mask=attention_mask)
        logits = logits.last_hidden_state

        logits = self.fc(logits)

        loss = None
        if labels is not None:
            shift_logits = logits[:, :-1].reshape(-1, tokenizer.vocab_size)
            shift_labels = labels[:, 1:].reshape(-1)

            loss = self.criterion(shift_logits, shift_labels)

        return {'loss': loss, 'logits': logits}


model = Model()

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

out = model(**data)

out['loss'], out['logits'].shape

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


12050.9952


(tensor(6.4238, grad_fn=<NllLossBackward0>), torch.Size([8, 8, 50257]))

In [5]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=8,
        collate_fn=default_data_collator,
        shuffle=True,
        drop_last=True,
    )

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):

        #只计算最后一个词的正确率,这里先把最后一个词取出来
        label = data['input_ids'][:, -1].clone()

        #从数据中抹除掉最后一个词,防止模型作弊
        data['input_ids'][:, -1] = 0

        #label就不需要了
        data['labels'][:, :] = 0

        #计算
        with torch.no_grad():
            out = model(**data)

        #只计算最后一个词的正确率,因为有偏移量的关系,这里取的是倒数第二个词
        out = out['logits'].argmax(dim=2)[:, -2]

        correct += (label == out).sum().item()
        total += 8

        if i % 10 == 0:
            print(i)
            print(label)
            print(out)

        if i == 50:
            break

    print(correct / total)

    for i in range(8):
        print(tokenizer.decode(data['input_ids'][i, :-1]))
        print(tokenizer.decode(label[i]), tokenizer.decode(out[i]))
        print()


test()

0
tensor([ 3159, 22152,  5321, 26810,   306,  1997,   318,   837])
tensor([  995,  1943, 13839,  2646,   306,   257,    13,    11])
10
tensor([  803,  2004, 16304,   996,   463,   973,    12,   257])
tensor([  803,    81,  2836,   290, 49191,   262,    12,   262])
20
tensor([39628,   287,   764,  7464,  1111,   356, 15827,  7357])
tensor([ 267,  287,   13, 1110,  290,  407,  290, 7357])
30
tensor([ 2646,   262,  3227,   262, 23372,   290,    82,   663])
tensor([ 13, 262,  13, 290, 265, 284,  82, 262])
40
tensor([  764,  1838,   475, 23365,   278,  2230,   422,   379])
tensor([   13,   318,   290, 23365,   278,   837,   422,   290])
50
tensor([ 351, 3732,  389,  318,  290, 3613,  581,  262])
tensor([  290, 42418,   284,   318,    11,  5368,   922,   262])
0.20833333333333334
plays as hollow catharsis,
 with  and

somewhat blurred, but k
inn udos

denzel washington's efforts
 are  to

a tale of horror and revenge that
 is  is

these spiders can outrun a motorcycle
 and ,

not even the ha

In [6]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    model.train()
    for i, data in enumerate(loader):
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            labels = data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]

            correct = (labels == out).sum().item()
            accuracy = correct / (8 * 7)

            lr = optimizer.state_dict()['param_groups'][0]['lr']

            print(i, loss.item(), accuracy, lr)

    torch.save(model, 'models/1.预测最后一个词.model')


train()

/root/anaconda3/envs/cpu/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 6.279179096221924 0.10714285714285714 1.999599037690457e-05
50 5.47908353805542 0.21428571428571427 1.979550922213312e-05
100 5.5193705558776855 0.16071428571428573 1.959502806736167e-05
150 4.97205114364624 0.23214285714285715 1.939454691259022e-05
200 5.653367042541504 0.19642857142857142 1.9194065757818766e-05
250 4.709580421447754 0.25 1.8993584603047316e-05
300 5.8663482666015625 0.16071428571428573 1.8793103448275863e-05
350 4.809518337249756 0.19642857142857142 1.8592622293504413e-05
400 4.790334224700928 0.26785714285714285 1.839214113873296e-05
450 5.267386436462402 0.23214285714285715 1.819165998396151e-05
500 4.75763463973999 0.26785714285714285 1.7991178829190057e-05
550 5.370026111602783 0.17857142857142858 1.7790697674418608e-05
600 4.932478427886963 0.2857142857142857 1.7590216519647154e-05
650 4.6484599113464355 0.26785714285714285 1.7389735364875705e-05
700 5.083922863006592 0.19642857142857142 1.718925421010425e-05
750 5.430644989013672 0.16071428571428573 1.6988773

In [7]:
model = torch.load('models/1.预测最后一个词.model')
test()

0
tensor([  268,   257,   837,   467, 31557,  3159,   326,  5730])
tensor([ 268,  329,  290,  257, 3437, 1295,  286, 5730])
10
tensor([27235,   318,  1628, 17548,   257,   837,  1621,   783])
tensor([ 837,  318,  837,  749,  705, 2003, 1621,  508])
20
tensor([2181,  318,  837,  768,  318,  286, 1628, 2085])
tensor([3360,  318, 7328,  768,  318, 1022, 3807, 1178])
30
tensor([  416, 15360,   468,   837,   837,  7464,   705,   326])
tensor([ 220,  257,  705,  220,  345, 3807,  705,  286])
40
tensor([ 290,  508, 1267,  290,  262,   12, 7675,  257])
tensor([ 837, 2073, 1267,  220,  262,   12,  705,  257])
50
tensor([ 1372,   837,   502,   428,   546,   340, 16728,    82])
tensor([262, 290, 262, 257, 326, 340, 286,  82])
0.2818627450980392
none of this violates the letter of
 beh  the

chicago is sophisticated, brash
,  and

they felt like the same movie to
 me  the

a recent favorite at sundance,
 this  a

` dragonfly'is a movie
 about  that

the problem with this film is that
 it  it

if y